## Learning Objectives

At the end of the experiment you will be able to :

- finetune the resnet model to determine a dog's breed from a given image


In [ ]:
#@title Experiment Explanation Video
from IPython.display import HTML

HTML("""<video width="850" height="480" controls>
  <source src="https://cdn.talentsprint.com/talentsprint1/archives/sc/misc/resnet50_dog_breed_classification.mp4" type="video/mp4">
</video>
""")

## Dataset

### Description


This dataset has been extracted from [stanford](http://vision.stanford.edu/aditya86/ImageNetDogs/main.html) which contains images of breeds of dogs from around the world where each image is a subset from ImageNet

There are around 1,000 images, out of which 850 are used for training and 150 for testing

The dataset comprises 12 breeds of dogs:

    african_hunting_dog
    beagle
    bloodhound
    chow
    doberman
    eskimo_dog
    german_shepherd
    golden_retriever
    leonberg
    lhasa
    pug
    redbone

### Transfer Learning

Transfer learning is a machine learning technique in which a network that has been trained to perform a specific task is being reused (repurposed) as a starting point for another similar task.

In [ ]:
! wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/dog_breed_images.zip")
! unzip dog_breed_images.zip")

### 1. Importing required packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchsummary import summary
from torchvision import datasets, transforms, models

### 2.  Data Preprocessing

In [ ]:
# Specify root data directory
data_dir = 'dog_breed_images'

batch_size = 10

# ResNet50 input is 224x224 by default
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [ ]:
transform = transforms.Compose([transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),transforms.ToTensor()])

# Loading the data
trainset = datasets.ImageFolder(data_dir + '/Train', transform=transform)

testset = datasets.ImageFolder(data_dir + '/Test', transform=transform)

# Load the data. utils.dataloader is a package for loading the dataset 
train_loader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size=batch_size)

In [ ]:
# Check number of training and test images
dataset_sizes = {'Train': len(trainset), 'Test': len(testset)}
dataset_sizes

In [ ]:
# Generate a batch of 10 images and labels
train_images, train_labels = next(iter(train_loader))
train_images.shape, train_labels.shape

### 3. Visualizing the train images

In [ ]:
# labels Translator 
label_names = {v: k for k, v in trainset.class_to_idx.items()}
label_names

In [ ]:
# Create a grid of images along with their corresponding labels
L = 3
W = 3

fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.reshape(-1)

for i in np.arange(0, L*W):
    axes[i].imshow(train_images[i].permute(1, 2, 0))
    axes[i].set_title(label_names[train_labels[i].item()])
    axes[i].axis('off')

plt.tight_layout()

### 4. Initializing CUDA

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### 5. Loading Resnet50 model with pretrained weights

In [ ]:
basemodel = models.resnet50(pretrained=True)
print(basemodel)

In [ ]:
print(basemodel.fc)

### 6. Fine-tuning ResNet-50 

* The first layers of Resnet50 are used to extract high level general features
* The last couple of layers are used to perform classification (on a specific task)
* Copy the first trained layers (base model) and then add a new custom layers in the output to perform classification on a specific task


In [ ]:
# Freeze all layers
for param in basemodel.parameters():
    param.requires_grad = False

# Parameters of the below newly constructed modules have "requires_grad=True" 
basemodel.fc = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(512, len(label_names)))

criterion = nn.CrossEntropyLoss()

# Optimize only the fully connected layer portion
optimizer = optim.SGD(basemodel.fc.parameters(), lr=0.003, momentum=0.5)
model = basemodel.to(device)

In [ ]:
# Print the summary of the model
from torchsummary import summary
summary(model, input_size=(3, IMG_HEIGHT, IMG_WIDTH))

### 7. Train the deep learning model

In [ ]:
def train(net, trainloader, trainset, epochs):
    # keeping the network in train mode
    net.train()

    train_loss,  train_accuracy = [], []
    # Loop for no of epochs
    for epoch in range(epochs+1):
          running_loss = 0.0
          running_accuracy = 0.0

          # Iterate through all the batches in each epoch
          for images, labels in (trainloader):
                  images, labels = images.to(device), labels.to(device)

                  #-----------------Forward Pass----------------------
                  outputs = net(images)
                  loss = criterion(outputs, labels) # Calculating the loss

                  #-----------------Backward Pass---------------------
                  optimizer.zero_grad() # Zero the parameter gradients
                  loss.backward()
                  optimizer.step() # update the weights accordingly

                  running_loss+=loss.item()
                  
                  # Accuracy calculation
                  _, predicted = torch.max(outputs, 1)
                  running_accuracy += (predicted == labels).sum().item()

          #-----------------Log-------------------------------
          loss = running_loss/len(trainset)
          train_loss.append(loss)

          accuracy = 100 * (running_accuracy/len(trainset))
          train_accuracy.append(accuracy)
          print("======> epoch: {}/{}, Train Loss:{:.4f} Train Accuracy:{:.2f}".format(epoch,epochs,loss,accuracy))
    return net, train_loss, train_accuracy

In [ ]:
model, train_loss, train_accuracy = train(model, train_loader, trainset, 5)

### 8. Evaluate the trained deep learning model

In [ ]:
def test(net, testloader, testset):
    # keeping the network in evluation mode
    net.eval()
    
    predicted_label, original_label, test_images = [], [], []
    running_accuracy = 0.0

    # Iterate through all the batches
    for images, labels in (testloader):
            images, labels = images.to(device), labels.to(device)

            #-----------------Forward Pass----------------------
            outputs = net(images)
            
            # Accuracy calculation
            _, predicted = torch.max(outputs, 1)
            running_accuracy += (predicted == labels).sum().item()
            
            test_images.extend(images.cpu())
            predicted_label.extend(predicted.cpu())
            original_label.extend(labels.cpu())

    #-----------------Log-------------------------------
    accuracy = 100 * (running_accuracy/len(testset))
    print("======> Test Accuracy:{:.2f}".format(accuracy))
    return accuracy, predicted_label, original_label, test_images

In [ ]:
test_accuracy, predicted_label, original_label, test_images = test(model, test_loader, testset)

### 9. Visualizing the test images along with the predictions

In [ ]:
# Create a grid of images along with their corresponding labels
L = 3
W = 3

fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.reshape(-1)

for i in np.arange(0, L*W):
    axes[i].imshow(test_images[i].permute(1, 2, 0))
    axes[i].set_title('\n{}\nPredicted: {}'.format(str(label_names[original_label[i].item()]),str(label_names[predicted_label[i].item()])))
    axes[i].axis('off')

plt.tight_layout()